In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType
from pyspark.sql.functions import expr, array, col, explode, arrays_zip, lit, avg, when, broadcast

# warehouse_location points to the default location for managed databases and tables
warehouse = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.4.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
jobs_df = spark.table("database.jobs")
popu_df = spark.table("database.population")
jobs_df = jobs_df.filter(col("indicador_name").like("%Labor force with basic education%"))
jobs_df = jobs_df.drop(col("pais_cod"), col("indicador_code"))
popu_df = popu_df.filter(col("cod_indicador").like("%SP.POP.TOTL%"))
popu_df = popu_df.drop(col("cod_pais"), col("cod_indicador"), col("nome_indicador"), col("Country_name"), col("Country_code"), col("Indicator_name"), col("Indicator_code"))
popu_df = popu_df.withColumnRenamed("valor", "valorI")
popu_df = popu_df.withColumnRenamed("nome_pais", "pais_name")
popu_df = popu_df.withColumn("ano", col("ano").cast(IntegerType()))

In [3]:
paises = ["%Liechtenstein%", "%Germany%", "%Denmark%", "%Switzerland%", "%Poland%", "%Czech%", 
          "%Slovakia%", "%Slovenia%", "%Hungary%", "%Austria%"]
anos = ["2010", "2011", "2012", "2013", "2014", "2015"]

jobs_df = jobs_df.filter(
    col("pais_name").like(paises[0]) |
    col("pais_name").like(paises[1]) |
    col("pais_name").like(paises[2]) |
    col("pais_name").like(paises[3])
).filter(
    col("ano").isin(anos) &
    col("valor").isNotNull()
)

popu_df = popu_df.filter(
    col("pais_name").like(paises[0]) |
    col("pais_name").like(paises[1]) |
    col("pais_name").like(paises[2]) |
    col("pais_name").like(paises[3])
).filter(
    col("ano").isin(anos) &
    col("valorI").isNotNull()
)

In [4]:
literacia_df = popu_df.join(jobs_df, ["pais_name", "ano"], "left")
literacia_df = literacia_df.withColumn("valorI", ((col("valor") / 100) * (col("valorI") / 2)).cast("int"))
literacia_df = literacia_df.withColumnRenamed("pais_name", "paises")

In [5]:
literacia_df = literacia_df.withColumn("genero",
    when(col("indicador_name").like("% female%"), "F")
    .when(col("indicador_name").like("% male%"), "M")
)

literacia_df = literacia_df.filter(col("genero").isNotNull())

literacia_df = literacia_df.drop(col("indicador_name"))

literacia_df.toPandas()

,paises,ano,valorI,valor,genero
0,Denmark,2010,1600367,57.695000,F
1,Denmark,2011,1569441,56.347599,F
2,Denmark,2012,1505343,53.843300,F
3,Denmark,2013,1465626,52.204601,F
4,Denmark,2014,1488805,52.762001,F
5,Denmark,2015,1485969,52.290798,F
6,Denmark,2010,1360225,49.037601,M
7,Denmark,2011,1328428,47.694500,M
8,Denmark,2012,1277769,45.703400,M
9,Denmark,2013,1238404,44.111099,M


In [6]:
literacia_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Projeto/gold/TabelaLabor/")

In [7]:
spark.sql(
    """
    SELECT * FROM database.LaborGold
    """
).show()

+-------+----+-------+--------+------+
| paises| ano|  valor|  valorI|genero|
+-------+----+-------+--------+------+
|Denmark|2010| 57.695| 1600367|     F|
|Denmark|2011|56.3476| 1569441|     F|
|Denmark|2012|53.8433| 1505343|     F|
|Denmark|2013|52.2046| 1465626|     F|
|Denmark|2014| 52.762| 1488805|     F|
|Denmark|2015|52.2908| 1485969|     F|
|Denmark|2010|49.0376| 1360225|     M|
|Denmark|2011|47.6945| 1328428|     M|
|Denmark|2012|45.7034| 1277769|     M|
|Denmark|2013|44.1111| 1238404|     M|
|Denmark|2014|43.9844| 1241124|     M|
|Denmark|2015|43.2067| 1227822|     M|
|Germany|2010|51.0883|20889220|     F|
|Germany|2011|57.4606|23063244|     F|
|Germany|2012|57.2197|23009707|     F|
|Germany|2013|57.4174|23152305|     F|
|Germany|2014|50.4682|20435204|     F|
|Germany|2015| 49.937|20395920|     F|
|Germany|2010|37.0592|15152937|     M|
|Germany|2011|40.3295|16187249|     M|
+-------+----+-------+--------+------+
only showing top 20 rows



In [8]:
spark.stop()